In [1]:
import urllib
import pandas as pd
from tqdm import tqdm


def read_csv_from_url(url):
    try:
        return pd.read_csv(url)
    except urllib.error.HTTPError as e:
        if e.status == 404:
            return None
        raise e


countries = pd.read_csv('../data/countries.csv', na_filter=False)

In [2]:
PLACES = ['parks', 'residential', 'retail-and-recreation', 'transit-stations', 'workplaces', 'grocery-and-pharmacy']


def google_mobility_by_country(code):
    data = []

    for place in PLACES:
        df = read_csv_from_url(f'https://pastelsky.github.io/covid-19-mobility-tracker/output/{code}/mobility-{place}.csv')
        if df is not None:
            df = df.rename(columns={'value': place})
            data.append(df)
    
    if data:
        return pd.concat(data).groupby('date').first()
    else:
        return None


data = []

for i, row in tqdm(countries[countries['iso_alpha2'] != ''][['country', 'iso_alpha2']].iterrows()):
    mobility = google_mobility_by_country(row['iso_alpha2'])

    if mobility is not None:
        mobility['country'] = row['country']
        data.append(mobility)

pd.concat(data).groupby(['country', 'date']).first().reset_index().to_csv('../data/mobility-google.csv', index=False)

168it [05:02,  1.80s/it]
